# AR(1) + GARCH(1,1) Model

## Loading Packages

In [1]:
using Optim

include("jlFiles/printmat.jl")

printmatDate

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

## Loading Data

In [3]:
xx  = readdlm("Data/FFdSizePs.csv",',',skipstart=1)
ymd = round.(Int,xx[:,1])     #YearMonthDay, like 20121231
R   = xx[:,2]                 #returns for the smallest size portfolio
xx  = nothing

y = R[2:end]                           #dependent variable, y(t)
x = [ones(size(R,1)-1) R[1:end-1]]     #regressors, [1, y(t-1)]

dN = Date.(string.(ymd),"yyyymmdd")
println("The first four dates:")
printmat(dN[1:4])

The first four dates:
1980-01-02
1980-01-03
1980-01-04
1980-01-07



## The Likelihood Function

Consider a regression equation, where the residual follows a GARCH(1,1) process

$$
\begin{align*}
y_{t}           &  =x_{t}^{\prime}b+u_{t} \ \text{ with }\ u_{t}=v_{t}\sigma_{t}\\
\sigma_{t}^{2}  &  =\omega+\alpha u_{t-1}^{2}+\beta\sigma_{t-1}^{2}.
\end{align*}
$$
Note that we require $(\omega,\alpha,\beta)$ to all be positive and $\alpha + \beta < 1$.

If $v_{t}\sim N(0,1)$, then the likelihood function is

$$
\ln L=-\frac{T}{2}\ln\left(  2\pi\right)  -\frac{1}{2}\sum_{t=1}
^{T}\ln\sigma_{t}^{2}-\frac{1}{2}\sum_{t=1}^{T}\frac{u_{t}^{2}}{\sigma_{t}
^{2}}.
$$

The likelihood function of a GARCH(1,1) model can be coded as in
*garch11LL*. The first function calculates time-varying  variances and
the likelihood contributions (for each period). The second functions forms the
loss function used in the minimization.

In [4]:
function garch11LL(par::Vector,y,x)

  (T,k) = size(x,1,2)

  b                   = par[1:k]             #mean equation, y = x'*b
  (omega,alpha,beta1) = par[k+1:k+3]         #GARCH(1,1) equation: 
                                             #s2(t) = omega + alpha*u(t-1)^2 + beta1*s2(t-1)        
  yhat = x*b
  u    = y - yhat
  s2_0 = var(u)                                 #var(u,1) gives a matrix, var(u) a scalar

  s2    = zeros(typeof(alpha),T)                 #works also with ForwardDiff 
  s2[1] = omega + alpha*s2_0 + beta1*s2_0        #simple, but slow approach
  for t = 2:T                                    #using filter() is perhaps quicker
    s2[t] = omega + alpha*u[t-1]^2 + beta1*s2[t-1]
  end

  LL    = -(1/2)*log(2*pi) .- (1/2)*log.(s2) .- (1/2)*(u.^2)./s2
  LL[1] = 0.0               #effectively skip the first observation

  return LL,s2,yhat

end


function garch11LLLoss(par::Vector,y,x)         #loss fn to minimize
    par1            = copy(par)
    par1[end-2:end] = abs.(par1[end-2:end])     #impose non-negativity on (omega,alpha,beta)
    LL,  = garch11LL(par1,y,x)
    Loss = -sum(LL)                             #to minimize: -sum(LL)
    return Loss
end


function garch11LLRLoss(par::Vector,y,x,rho)     #loss fn, with penalty on alpha+beta1 > 1
    par1            = copy(par)
    par1[end-2:end] = abs.(par1[end-2:end])      #impose non-negativity
    LL, = garch11LL(par1,y,x)
    (alpha,beta1) = par1[end-1:end]   #s2(t) = omega + alpha*u(t-1)^2 + beta1*s2(t-1)
    g = [alpha + beta1 - 1]                     #alpha + beta1 < 1
    Loss = -sum(LL) + rho*sum(max.(0,g).^2)
    return Loss
end

garch11LLRLoss (generic function with 1 method)

## Try the Likelihood Function

In [5]:
par0 = [mean(y);0;var(y)*0.05;0.05;0.93]         #initial parameter guess

(loglik,s2,yhat) = garch11LL(par0,y,x)            #just testing the log lik
LL               = garch11LLLoss(par0,y,x)
printlnPs("Value of log-likelihood fn at starting guess: ",-LL)

Value of log-likelihood fn at starting guess:  -9846.688


## Maximize the Likelihood Function

In [6]:
Sol = optimize(par->garch11LLLoss(par,y,x),par0)   #minimize -sum(LL)

parHat            = Optim.minimizer(Sol)          #extract the optimal solution 
parHat[end-2:end] = abs.(parHat[end-2:end])       #since the likelihood function uses abs(these values)

LLHat             = garch11LLLoss(parHat,y,x)
printlnPs("Value of log-likelihood fn at estimate: ",-LLHat)

println("\nParameter estimates (b[1],b[2],omega,alpha,beta1): ")
printmat(parHat)

Value of log-likelihood fn at estimate:  -8243.608

Parameter estimates (b[1],b[2],omega,alpha,beta1): 
     0.049
     0.256
     0.014
     0.162
     0.824



## Standard Errors of the Estimates

MLE is typically asymptotically normally distributed
$$
\begin{align*}
\sqrt{T}(\hat{\theta}-\theta)  &  \rightarrow^{d}N(0,V)\text{, where } V=I(\theta)^{-1}\text{ with }\\
I(\theta)  &  =-\operatorname*{E}\frac{\partial^{2}\ln L_t}{\partial\theta\partial\theta^{\prime}}
\end{align*}
$$
where $\ln L_t$  is the contribution of period $t$ to the likelihood function and $I(\theta)$ is the information matrix.

The code below calculates numerical derivatives.  


Alternatively, we can use the outer product of the gradients to calculate the
information matrix as
$$
J(\theta)=\operatorname*{E}\left[  \frac{\partial\ln L_t}{\partial\theta
}\frac{\partial\ln L_t}{\partial\theta^{\prime}}\right]
$$

We could also use the "sandwich" estimator

$$
V=I(\theta)^{-1}J(\theta)I(\theta)^{-1}.
$$

### Std from Hessian

In [7]:
T = size(y,1)                    #finding std(coefs) by inverse of information matrix

Ia         = -ForwardDiff.hessian(par->mean(garch11LL(par,y,x)[1]),parHat)
Ia         = (Ia+Ia')/2              #to guarantee symmetry
vcv        = inv(Ia)/T
std_parHat = sqrt.(diag(vcv))

println("std from Hessian")
printmat(std_parHat)

std from Hessian
     0.006
     0.012
     0.001
     0.009
     0.009



### Std from Gradient and Sandwich

In [8]:
LLgrad = ForwardDiff.jacobian(par->garch11LL(par,y,x)[1],parHat)   #T x length(par) matrix, T gradients
J           = LLgrad'LLgrad/T
vcv         = inv(J)/T
stdb_parHat = sqrt.(diag(vcv))                          #std from gradients

vcv         = inv(Ia) * J * inv(Ia)/T
stdc_parHat = sqrt.(diag(vcv))                          #std from sandwich

printlnPs("\nGARCH parameter estimates and 3 different standard errors
    coef     hessian     gradient  sandwich")
printmat([parHat std_parHat stdb_parHat stdc_parHat])


GARCH parameter estimates and 3 different standard errors
    coef     hessian     gradient  sandwich
     0.049     0.006     0.006     0.007
     0.256     0.012     0.012     0.013
     0.014     0.001     0.001     0.003
     0.162     0.009     0.005     0.020
     0.824     0.009     0.005     0.019



## Redo MLE, but with Penalty on alpha+beta>1

In [9]:
println("\nIterate with harder and harder punishment on α + β < 1 restriction. PERHAPS NOT NEEDED")

println("\n    penalty       b0      b1       omega     alpha     beta")
options = Optim.Options(show_trace=false,show_every=10)
for rho = 0.0:0.1:1
  Sol = optimize(par->garch11LLRLoss(par,y,x,rho),par0,options)
  par = Optim.minimizer(Sol)
  par[end-3:end-1] = abs.(par[end-3:end-1])
  printmat([rho par'])
  par0 = copy(par)
end


Iterate with harder and harder punishment on α + β < 1 restriction. PERHAPS NOT NEEDED

    penalty       b0      b1       omega     alpha     beta
     0.000     0.049     0.256     0.014     0.162     0.824

     0.100     0.049     0.256     0.014     0.162     0.824

     0.200     0.049     0.256     0.014     0.162     0.824

     0.300     0.049     0.256     0.014     0.162     0.824

     0.400     0.049     0.256     0.014     0.162     0.824

     0.500     0.049     0.256     0.014     0.162     0.824

     0.600     0.049     0.256     0.014     0.162     0.824

     0.700     0.049     0.256     0.014     0.162     0.824

     0.800     0.049     0.256     0.014     0.162     0.824

     0.900     0.049     0.256     0.014     0.162     0.824

     1.000     0.049     0.256     0.014     0.162     0.824



# Value at Risk

calculated by assuming conditional (time-varying) normality,

$
\mathrm{VaR} = -(\mu_t - 1.645 \sigma_t),
$

where 
$\mu_t$ are the predictions from the estimated mean equation ($x_t'b$) and $\sigma_t$ from the GARCH(1,1) model.

In [10]:
(_,σ²,μ) = garch11LL(parHat,y,x)
VaR95     = -(μ - 1.645*sqrt.(σ²))

CovRatio = mean((-y) .>= VaR95)             #coverage ratio for VaR
printlnPs("\nCoverage ratio for VaR(95%): ",CovRatio)


Coverage ratio for VaR(95%):      0.058


In [11]:
xTicks = [Date(1990);Date(2000);Date(2010)]     #controlling the tick locations

plot(dN[2:end],VaR95,xticks=Dates.value.(xTicks),legend=false)
title!("1-day VaR (95%)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1990-01-01 
 
 
 2000-01-01 
 
 
 2010-01-01 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 1-day VaR (95%) 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 28.9273,327.277 28.973,323.469 29.1103,337.354 29.1561,334.409 29.2018,339.631 29.2476,338.248 29.2933,343.322 29.4306,342.213 29.4763,346.993 29.5221,346.718 
 29.5679,349.849 29.6136,351.069 29.7509,354.77 29.7966,357.896 29.8424,342.157 29.8881,353.681 29.9339,355.924 30.0712,357.386 30.1169,359.834 30.1627,349.015 
 30.2084,357.281 30.2542,355.006 30.3914,359.15 30.4372,359.33 30.4829,362.51 30.5287,365.848 30.5745,367.113 30.7117,367.179 30.7575,355.367 30.8032,361.565 
 30.849,364.032 30.8947,346.915 31.0778,350.982 31.1235,352.214 31.1693,361.23 31.215,352.653 31.3523,353.092 31.398,345.903 31.4438,357.561 31.4895,356.53 
 31.5353,360.396 31.6726,364.224 31.7183,353.757 31.7641,348.762 31.8098,336.058 31.8556,298.922 31.9928,310.955 32.0386,300.813 32.0844,326.107 32.1301,327.084 
 32.1759,330.865 32.3131,335.668 32.3589,286.836 32.4046,307.626 32.4504,327.083 32.4961,321.253 32.6334,327.285 32.6792,281.069 32.7249,288.678 32.7707,310.286 
 32.8164,167.115 32.9537,228.914 32.9994,223.454 33.0452,235.545 33.091,251.883 33.274,255.659 33.3197,246.112 33.3655,273.818 33.4112,289.546 33.457,299.305 
 33.5943,304.183 33.64,300.211 33.6858,311.172 33.7315,313.63 33.7773,315.154 33.9145,328.913 33.9603,320.149 34.006,332.614 34.0518,328.868 34.0976,333.932 
 34.2348,326.888 34.2806,337.312 34.3263,342.503 34.3721,345.48 34.4178,344.282 34.5551,351.104 34.6009,354.88 34.6466,357.963 34.6924,361.059 34.7381,355.267 
 34.8754,357.112 34.9211,360.201 34.9669,364.999 35.0126,366.329 35.0584,366.102 35.1957,366.962 35.2414,367.907 35.2872,368.077 35.3329,368.435 35.3787,370.823 
 35.5617,369.532 35.6075,365.971 35.6532,366.945 35.699,359.378 35.8362,365.597 35.882,364.832 35.9277,365.825 35.9735,369.259 36.0192,367.243 36.1565,368.481 
 36.2023,368.353 36.248,369.71 36.2938,371.553 36.3395,365.768 36.4768,370.087 36.5225,368.091 36.5683,369.497 36.614,369.602 36.6598,366.799 36.7971,369.832 
 36.8428,372.613 36.8886,373.014 36.9343,372.664 36.9801,367.815 37.1173,370.554 37.1631,356.234 37.2089,364.398 37.2546,367.754 37.4376,368.145 37.4834,365.503 
 37.5291,363.606 37.5749,367.691 37.6206,363.849 37.7579,368.132 37.8037,366.984 37.8494,361.852 37.8952,364.621 37.9409,365.503 38.0782,364.23 38.1239,364.141 
 38.1697,358.042 38.2155,362.971 38.2612,366.316 38.3985,364.173 38.4442,367.771 38.49,369.942 38.5357,369.205 38.5815,360.426 38.7188,365.67 38.7645,361.974 
 38.8103,366.139 38.856,368.332 38.9018,368.076 39.039,365.447 39.0848,367.287 39.1305,357.114 39.1763,363.706 39.2221,363.836 39.3593,360.762 39.4051,338.626 
 39.4508,347.871 39.4966,356.381 39.5423,356.808 39.6796,354.43 39.7254,352.084 39.7711,356.358 39.8169,355.697 39.8626,352.402 40.0456,360.41 40.0914,361.81 
 40.1371,361.709 40.1829,355.224 40.3202,359.766 40.3659,353.092 40.4117,360.459 40.4574,362.992 40.5032,362.099 40.6404,361.63 40.6862,361.011 40.732,363.437 
 40.7777,362.795 40.8235,354.927 40.9607,360.028 41.0065,355.301 41.0522,342.012 41.098,352.989 41.1437,353.445 41.281,323.896 41.3268,295.958 41.3725,323.95 
 41.4183,324.434 41.464,330.904 41.6013,337.08 41.647,339.335 41.6928,335.733 41.7386,343.772 41.7843,343.737 41.9216,348.345 41.9673,353.254 42.0131,350.967 
 42.0588,355.78 42.1046,342.914 42.2419,352.222 42.2876,356.592 42.3334,354.208 42.3791,361.252 42.4249,343.922 42.5621,355.303 42.6079,344.171 42.6536,349.664 
 42.6994,357.412 42.7452,348.683 42.8824,357.329 42.9739,360.716 43.0197,362.674 43.0654,335.271 43.2027,346.268 43.2485,350.699 43.2942,357.41 43.34,357.765 
 43.3857,356.4 43.523,355.946 43.5687,352.741 43.6145,358.364 43.6602,358.861 43.706,361.787 43.8433,354.435 43.889,351.229 43.9348,360.092 44.0